# Generate CMOD dataset

In [6]:
import os
os.getcwd()

'/home/yumouwei/2025_wm_summer_school/2025-wm-summer-school-data'

In [1]:
import pandas as pd
import disruption_py

In [10]:
# read shotlists

disruption_shotlist = pd.read_csv('./shotlists/disruption_shotlist.txt', header=None)
disruption_shotlist = disruption_shotlist[0].astype(int).to_list()
non_disruptive_shotlist = pd.read_csv('./shotlists/non_disruptive_shotlist.txt', header=None)
non_disruptive_shotlist = non_disruptive_shotlist[0].astype(int).to_list()

In [ ]:
print(f"Number of disruptive shots: {len(disruption_shotlist)}")
print(f"Number of non disruptive shots: {len(non_disruptive_shotlist)}")

3795